In [231]:
import numpy as np
import scipy.special
import matplotlib.pyplot
import json
%matplotlib inline

In [232]:
class NeuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # creating weights
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        self.lr = learningrate

        self.activation_function = lambda x: scipy.special.expit(x)
        
    
    def query(self, input_list):
        # convert input list to 2d array
        inputs = np.array(input_list, ndmin=2).T  # Bogdan Clarify why we need to do it
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # caclucate signals into output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

    def train(self, input_list, targets_list):
        # convert inputs to 2d array
        inputs = np.array(input_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate signals of outputs
        final_outputs = self.activation_function(final_inputs)
        
        # calculating errors (target - actual)
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weights for the links between hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weights between input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    def print_weights(self):
        print('====weights input hidden====')        
        print('Size: ', self.wih.size)
        print(len(self.wih[0]))
        print(self.wih[0])
        print('====weights hidden output====')
        print('Size: ', self.who[0].size)
        print(self.who[0])
    
    def save_weights_to_file(self, filename='saved_weights.npz'):
        np.savez(filename, wih=self.wih, who=self.who)
        print('Saved weights to file: "{}"'.format(filename))
    
    def load_weights_from_file(self, filename='saved_weights.npz'):
        '''
        Loads wih and who from .npz file. Loaded values are reassigned to:
        self.wih and self.who
        '''
        data = np.load(filename)
        self.wih, self.who = data['wih'], data['who']


In [233]:
input_nodes = 784 # Because we have picture 28x28 pixels
hidden_nodes = 100 # We can think of this number as representation of features that correspond to 
# picture of number. The more hidden_nodes more features our NN can see. If we choose smaller "hidden_nodes"
# that some of features should be combined 
output_nodes = 10 # Because we have numbers from 0 to 9

learning_rate = 0.3

n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# # with open('mnist_train_100.csv', 'r') as f:
# with open('mnist_train.csv', 'r') as f:
#         training_data_list = f.readlines()

# for record in training_data_list:
#     record = record.rstrip()
#     all_values = record.split(',')
#     # we need to scale inputs so, they would be small. 
#     inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
#     # now we need the targets. all_values[0] represents actual number
#     targets = np.zeros(output_nodes) + 0.01 # created [0.01, 0.01... 0.01]
#     # one of targets should correspond to our number. For example, if all_values[0]=='1',
#     # then targets = [0.01, 0.99, 0.01 ... 0.01]
#     targets[int(all_values[0])] = 0.99
    
#     n.train(inputs, targets)


In [234]:
with open('mnist_test.csv', 'r') as f:
        test_data_list = f.readlines()
# test_record = test_data_list[0].strip()
# test_record = test_record.split(',')
# image_array = np.asfarray(test_record[1:]).reshape(28, 28)
# matplotlib.pyplot.imshow(image_array, cmap='Greys', interpolation='None')

In [235]:
n.load_weights_from_file()

In [236]:
# Calulating correct guess percentage:
scorecard = []
for record in test_data_list:
    record = record.rstrip()
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (np.asfarray(all_values[1:]) / 255 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = np.argmax(outputs)
    if label == correct_label:
        scorecard.append(1)
    else: 
        scorecard.append(0)
scorecard_array = np.asarray(scorecard)
print('Correct guess percentage: ', scorecard_array.sum() / scorecard_array.size)
print('Number of correctly guessed labels:', scorecard_array.sum())
print('Size of test data: {} records'.format(scorecard_array.size))

Correct guess percentage:  0.9456
Number of correctly guessed labels: 9456
Size of test data: 10000 records


In [237]:
# n.print_weights()
# n.save_weights_to_file()